In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 308 kB 3.8 MB/s 
     |████████████████████████████████| 209 kB 43.5 MB/s 
     |████████████████████████████████| 81 kB 8.2 MB/s 
     |████████████████████████████████| 78 kB 5.7 MB/s 
     |████████████████████████████████| 49 kB 3.6 MB/s 
     |████████████████████████████████| 147 kB 45.2 MB/s 
     |████████████████████████████████| 112 kB 48.8 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=9f3908345ab27bce11ea0660e0b327481dedb5bc92d54cb64665a9292ca95373
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [ ]:
import pandas as pd
import numpy as np
from sklearn import datasets, ensemble
import optuna
from sklearn import datasets, ensemble
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm import tqdm
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [ ]:
path = '/content/drive/MyDrive/Datasets/lg_aimers/lg aimers/'

train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')
submission = pd.read_csv(path + 'sample_submission.csv')

In [ ]:
target_list = ['Y_01', 'Y_02', 'Y_03', 'Y_04', 'Y_05', 'Y_06',
       'Y_07', 'Y_08', 'Y_09', 'Y_10', 'Y_11', 'Y_12', 'Y_13', 'Y_14']

In [ ]:
for i in tqdm(range(0,3)):
    globals()['train_{}'.format(target_list[i])] = train[['X_01', 'X_02', 'X_03', 'X_04', 'X_05', 'X_06', 'X_07', 'X_08',
       'X_09', 'X_10', 'X_11', 'X_12', 'X_13', 'X_14', 'X_15', 'X_16', 'X_17',
       'X_18', 'X_19', 'X_20', 'X_21', 'X_22', 'X_23', 'X_24', 'X_25', 'X_26',
       'X_27', 'X_28', 'X_29', 'X_30', 'X_31', 'X_32', 'X_33', 'X_34', 'X_35',
       'X_36', 'X_37', 'X_38', 'X_39', 'X_40', 'X_41', 'X_42', 'X_43', 'X_44',
       'X_45', 'X_46', 'X_47', 'X_48', 'X_49', 'X_50', 'X_51', 'X_52', 'X_53',
       'X_54', 'X_55', 'X_56', f'{target_list[i]}']]
    locals()['train_{}'.format(target_list[i])].drop(columns=['X_48', 'X_47', 'X_23', 'X_04'], inplace=True)

  0%|          | 0/3 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
100%|██████████| 3/3 [00:00<00:00, 38.02it/s]


In [ ]:
X = train_Y_01.loc[:, 'X_01':'X_56']
y = train_Y_01.loc[:, 'Y_01':]

In [ ]:
target = test[X.columns]

In [ ]:
def objective_cat(trial):
    """
    Objective function to tune a `CatBoostRegressor` model.
    """

    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    params = {
        'loss' : 'squared_error',
        'min_samples_split':trial.suggest_int("min_samples_split", 3, 20),
        'min_samples_leaf':trial.suggest_int("min_samples_leaf", 3, 20),
        'n_estimators':trial.suggest_int("n_estimators", 3, 2000),
        'learning_rate' : trial.suggest_uniform('learning_rate',0.01, 1),
        'subsample' : trial.suggest_uniform('subsample',0.1, 1),
        'criterion' : 'friedman_mse',
        'max_depth': trial.suggest_int('max_depth',1, 15),
        'warm_start' : True,
        'verbose' : 0,
        'random_state' : 42


    }


    model = ensemble.GradientBoostingRegressor(
        # objective = 'MultiRMSE',
        **params,
    )
    model.fit(x_train, y_train)

    pred = model.predict(x_test)

    rmse = mean_squared_error(y_test, pred)


    return rmse

In [ ]:
study_catboost = optuna.create_study(direction="minimize")
study_catboost.optimize(objective_cat, n_trials=30)

[I 2022-08-09 08:24:41,911] A new study created in memory with name: no-name-e9d22ec0-72c8-48b4-9e66-504d2d1e4f63


[08:24:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-08-09 08:24:56,521] Trial 0 finished with value: 0.12262789449360724 and parameters: {'min_samples_split': 12, 'min_samples_leaf': 10, 'n_estimators': 240, 'learning_rate': 0.078664635282602, 'subsample': 0.8812409644488226, 'max_depth': 3}. Best is trial 0 with value: 0.12262789449360724.


[08:24:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-08-09 08:25:56,204] Trial 1 finished with value: 0.16773076643079488 and parameters: {'min_samples_split': 15, 'min_samples_leaf': 19, 'n_estimators': 170, 'learning_rate': 0.2529672907462705, 'subsample': 0.38857883477214916, 'max_depth': 12}. Best is trial 0 with value: 0.12262789449360724.


...

[I 2022-08-09 08:55:09,049] Trial 27 finished with value: 0.12413244623315497 and parameters: {'min_samples_split': 9, 'min_samples_leaf': 20, 'n_estimators': 877, 'learning_rate': 0.17801634172269032, 'subsample': 0.8196294087311325, 'max_depth': 2}. Best is trial 0 with value: 0.12262789449360724.


[08:55:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-08-09 08:56:29,969] Trial 28 finished with value: 0.12812992446033963 and parameters: {'min_samples_split': 12, 'min_samples_leaf': 17, 'n_estimators': 552, 'learning_rate': 0.11038492981944682, 'subsample': 0.9136641262278582, 'max_depth': 7}. Best is trial 0 with value: 0.12262789449360724.


[08:56:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2022-08-09 08:57:08,896] Trial 29 finished with value: 0.17010011560250382 and parameters: {'min_samples_split': 5, 'min_samples_leaf': 18, 'n_estimators': 142, 'learning_rate': 0.277236462714216, 'subsample': 0.4512426841669581, 'max_depth': 10}. Best is trial 0 with value: 0.12262789449360724.


In [ ]:
params=study_catboost.best_params 

In [ ]:
# params=study_catboost.best_params 
# params['objective'] = 'MultiRMSE'
# params['random_state'] = 42 

In [ ]:
params=study_catboost.best_params 
params['loss'] = 'squared_error'
params['random_state'] = 42 

In [ ]:
kf = KFold(n_splits = 5, random_state = 42, shuffle = True)

In [ ]:
NRMSES = []
cb_pred = np.zeros(target.shape[0])
for i, idx in enumerate(kf.split(X, y)) :
    tr_x, tr_y = X.loc[idx[0]], y.loc[idx[0]]
    val_x, val_y = X.loc[idx[1]], y.loc[idx[1]]
    
    cb = ensemble.GradientBoostingRegressor(**params)
    cb.fit(tr_x, tr_y)
    
    val_pred = cb.predict(val_x)
    NRMSE = mean_squared_error(val_y, val_pred, squared=False)
    print(f"{i + 1} Fold NRMSE = {NRMSE}")
    NRMSES.append(NRMSE)
    
    fold_pred = cb.predict(target) / kf.n_splits
    cb_pred += fold_pred

[09:01:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
1 Fold NRMSE = 0.35042614987580833
[09:02:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
2 Fold NRMSE = 0.34633349100749633
[09:02:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
3 Fold NRMSE = 0.3410720547358708
[09:02:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
4 Fold NRMSE = 0.3481742141675928
[09:03:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
5 Fold NRMSE = 0.34994203205254176


In [ ]:
np.mean(NRMSES)

0.347189588367862